Copyright (c) 2023 by the LEL-A team

This software is distributed under the terms of the MIT license
which is available at https://opensource.org/licenses/MIT

In [1]:
import json
import re
import random
import uuid
import pandas as pd

In [2]:
!mkdir -p src_data

In [3]:
# -nc option: do not download if available
!wget -nc -P src_data https://germanquad.s3.amazonaws.com/GermanDPR.zip

--2023-04-07 23:38:44--  https://germanquad.s3.amazonaws.com/GermanDPR.zip
Resolving germanquad.s3.amazonaws.com (germanquad.s3.amazonaws.com)... 54.231.232.161, 54.231.226.225, 3.5.29.226, ...
Connecting to germanquad.s3.amazonaws.com (germanquad.s3.amazonaws.com)|54.231.232.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20114171 (19M) [application/zip]
Saving to: ‘src_data/GermanDPR.zip’

GermanDPR.zip       100%[===================>]  19.18M  5.61MB/s    in 3.8s    

2023-04-07 23:38:49 (5.04 MB/s) - ‘src_data/GermanDPR.zip’ saved [20114171/20114171]



In [4]:
# -n option: do not unzip if already available
!unzip -n ./src_data/GermanDPR.zip -d src_data

Archive:  ./src_data/GermanDPR.zip
  inflating: src_data/GermanDPR/GermanDPR_test.json  
  inflating: src_data/GermanDPR/GermanDPR_test_in_squad_format.json  
  inflating: src_data/GermanDPR/GermanDPR_train.json  


In [5]:
DATASET_NAME_AND_VERSION = "InstructGermanDPR_v1"

In [6]:
random.seed(42)

In [7]:
# regex to remove Wikipedia headers
regex = re.compile(r"^.*\n*=+ *.+ *=+ *\n")

In [8]:
with open("./src_data/GermanDPR/GermanDPR_train.json", encoding="utf-8") as f:
    germandpr_train = json.load(f)

In [9]:
with open("./src_data/GermanDPR/GermanDPR_test.json", encoding="utf-8") as f:
    germandpr_test = json.load(f)

In [10]:
def create_synonyms():
    """Select synonyms from synonyms dict."""
    return {k: random.choice(v) for (k, v) in synonyms.items()}

In [11]:
def generate_positive_pair(question, ctx, answer):
    # do this once to be consistent for input and output
    synonyms = create_synonyms()
    
    input_result = random.choice(input_templates)
    input_result = input_result.format(**synonyms, question=question, ctx=ctx)
    
    output_result = random.choice(positive_output_templates)
    output_result = output_result.format(**synonyms, answer=answer)
    
    return input_result, output_result

In [12]:
def generate_negative_pair(question, ctx):
    # do this once to be consistent for input and output
    synonyms = create_synonyms()
    
    input_result = random.choice(input_templates)
    input_result = input_result.format(**synonyms, question=question, ctx=ctx)
    
    output_result = random.choice(negative_output_templates)
    output_result = output_result.format(**synonyms)
    
    return input_result, output_result

In [13]:
synonyms = {
    "frage": ["Frage", "Fragestellung", "Problemstellung", "Anfrage"],
    "text": ["Text", "Absatz", "Textbereich", "Abschnitt"],
    "exakt": ["exakt", "genau", "präzise", "akkurat"],
    "textstelle": ["Textstelle", "Textpassage", "Sequenz"],
    "ist": ["ist", "lautet", "heißt"],
    "antwort": ["Antwort", "Lösung"],
    "vorhanden": ["vorhanden", "verfügbar"],
    "antwort_nicht_gefunden": [
        "Antwort nicht gefunden.", 
        "Lösung nicht gefunden.",
        "Keine Antwort gefunden.", 
        "Keine Lösung gefunden.",        
        "Die Antwort wurde nicht gefunden.",
        "Die Lösung wurde nicht gefunden.",
    ],
}

In [14]:
input_templates = [
"""\
Finde die {antwort} auf die {frage}: {question}
Im folgenden {text}:

{ctx}

Gebe {exakt} die gefundene {textstelle} wieder oder sage "{antwort_nicht_gefunden}", wenn Du nichts gefunden hast.""",
    
"""\
Ich habe folgende {frage}: {question}
Finde in folgendem {text} die {antwort} und gib die gefundene {textstelle} {exakt} wieder.

{ctx}

Wenn nichts gefunden wurde, antworte mit: {antwort_nicht_gefunden}""",

"""\
Ich habe eine {frage} zu folgendem {text}:

{ctx}

Die {frage} lautet: {question}
Bitte finde im {text} die {antwort} und gib {exakt} nur diese {textstelle} wieder.
Informiere mich mit "{antwort_nicht_gefunden}", wenn nichts gefunden wurde.""",
    
"""\
Gegeben ist folgender {text}:

{ctx}

Meine {frage} dazu lautet: {question}
Bitte suche im {text} die {antwort} und schreibe {exakt} diese {textstelle}.
Antworte mit "{antwort_nicht_gefunden}", wenn Du nichts passendes gefunden hast.""",    
]

In [15]:
positive_output_templates = [
"Die {antwort} auf die {frage} {ist}: \"{answer}\"",
"Die {antwort} {ist}: \"{answer}\"",
"\"{answer}\" {ist} die {antwort}.",
"\"{answer}\" {ist} das Ergebnis.",
"{answer}",
]

In [16]:
negative_output_templates = [
"{antwort_nicht_gefunden}",
]

In [17]:
def generate_prompts(germandpr):
    """Iterate dataset and generate input and output prompts."""
    input_prompts = []
    outputs = []

    for i, g in enumerate(germandpr):
        question = g["question"].strip()
        assert isinstance(question, str)
        assert len(question) > 0

        answers = g["answers"]
        assert isinstance(answers, list)
        assert len(answers) == 1

        answer = answers[0].strip()  # .replace("''", "\"")
        assert isinstance(answer, str)
        assert len(answer) > 0

        positive_ctxs = g["positive_ctxs"]
        assert isinstance(positive_ctxs, list)
        assert len(positive_ctxs) == 1

        positive_ctx_text = positive_ctxs[0]["text"].strip()  # .replace("''", "\"")
        assert isinstance(positive_ctx_text, str)
        assert len(positive_ctx_text) > 0

        positive_ctx_text_clean = re.sub(regex, "", positive_ctx_text)
        positive_ctx_text_clean = positive_ctx_text_clean.strip()
        assert "==" not in positive_ctx_text_clean
        assert len(positive_ctx_text_clean) > 0

        if answer not in positive_ctx_text_clean:
            # we skip and print datasets where answer is not in positive context
            print("Anwer not found in context!")
            print(f"question: {question}\nanswer: {answer}\npositive_ctx_text_clean: {positive_ctx_text_clean}\n\n")
        else:    
            positive_input, positive_output = generate_positive_pair(question, positive_ctx_text_clean, answer)
            input_prompts.append(positive_input)
            outputs.append(positive_output)

        negative_ctxs = g["negative_ctxs"]
        assert isinstance(negative_ctxs, list)
        assert len(negative_ctxs) == 0

        hard_negative_ctxs = g["hard_negative_ctxs"]
        assert isinstance(hard_negative_ctxs, list)
        assert len(hard_negative_ctxs) > 0

        hard_negative_ctx_text_clean_list = []

        for hard_negative_ctx in hard_negative_ctxs:
            hard_negative_ctx_text = hard_negative_ctx["text"].strip()  # .replace("''", "\"")
            assert isinstance(positive_ctx_text, str)
            assert len(positive_ctx_text) > 0   

            hard_negative_ctx_text_clean = re.sub(regex, "", hard_negative_ctx_text)
            hard_negative_ctx_text_clean = hard_negative_ctx_text_clean.strip()
            assert "==" not in hard_negative_ctx_text_clean, hard_negative_ctx_text
            assert len(hard_negative_ctx_text_clean) > 0

            hard_negative_ctx_text_clean_list.append(hard_negative_ctx_text_clean)

        assert len(hard_negative_ctx_text_clean_list) > 0

        negative_input, negative_output = generate_negative_pair(question, random.choice(hard_negative_ctx_text_clean_list))
        input_prompts.append(negative_input)
        outputs.append(negative_output)
    
    return input_prompts, outputs

In [18]:
# generate train pompts
inpus_train, outputs_train = generate_prompts(germandpr_train)
assert len(inpus_train) == len(outputs_train)

Anwer not found in context!
question: Welche Spielkonsole wird am häufigsten ohne Lizenz nachgebaut?
answer: Nintendo_Entertainment_System
positive_ctx_text_clean: Das NES zählt zu den am meisten unlizenziert nachgebauten Spielkonsolen aller Zeiten. Als Sammelbegriff für diese Konsolen hat sich das Wort „Famiclone“ durchgesetzt, welches aus „Famicom“ (der japanischen NES-Version) und „Clone“ (Nachbau, Klon) abgeleitet ist. Selbst im Jahre 2010, also 25 Jahre nach Erscheinen der Konsole in den USA und trotz dessen, dass wesentlich leistungsfähigere 3D-Spielkonsolen in ihrer vierten Generation verfügbar sind, werden NES-kompatible Spielkonsolen weiterhin hergestellt. Da einige NES-Patente in den USA bereits 2005 abgelaufen sind, können dort nun legal sogenannte „Famiclones“ vertrieben werden, insofern diese keine urheberrechtlich geschützten Spiele enthalten, da deren Urheberrechtsschutz noch immer andauert. Zu den offiziell als NES-kompatibel vertriebenen Konsolen zählen unter anderem „

In [19]:
# generate test pompts
inpus_test, outputs_test = generate_prompts(germandpr_test)
assert len(inpus_test) == len(outputs_test)

# Explore Data

In [20]:
def print_train_example(index):
    print(inpus_train[index])
    print(outputs_train[index])

In [21]:
print_train_example(0)

Finde die Antwort auf die Frage: Wie viele christlichen Menschen in Deutschland glauben an einen Gott?
Im folgenden Text:

Eine Zusammenfassung von Umfrageergebnissen aus verschiedenen Staaten ergab im Jahr 2007, dass es weltweit zwischen 505 und 749 Millionen Atheisten und Agnostiker gibt. Laut der Encyclopædia Britannica gab es 2009 weltweit 640 Mio. Nichtreligiöse und Agnostiker (9,4 %), und weitere 139 Mio. Atheisten (2,0 %), hauptsächlich in der Volksrepublik China.
Bei einer Eurobarometer-Umfrage im Jahr 2005 wurde festgestellt, dass 52 % der damaligen EU-Bevölkerung glaubt, dass es einen Gott gibt. Eine vagere Frage nach dem Glauben an „eine andere spirituelle Kraft oder Lebenskraft“ wurde von weiteren 27 % positiv beantwortet. Bezüglich der Gottgläubigkeit bestanden große Unterschiede zwischen den einzelnen europäischen Staaten. Die Umfrage ergab, dass der Glaube an Gott in Staaten mit starkem kirchlichen Einfluss am stärksten verbreitet ist, dass mehr Frauen (58 %) als Männer 

In [22]:
print_train_example(1)

Gegeben ist folgender Text:

Der Glaube an eine ''Kommende Welt'' (Olam Haba) bzw. an eine ''Welt des ewigen Lebens'' ist ein Grundprinzip des Judentums. Dieser jüdische Glaube ist von dem christlichen Glauben an das ''Ewige Leben'' fundamental unterschieden. Die jüdische Lehre spricht niemandem das Heil dieser kommenden Welt ab, droht aber auch nicht mit Höllenstrafen im Jenseits. Juden glauben schlicht, dass allen Menschen ein Anteil der kommenden Welt zuteilwerden kann. Es gibt zwar viele Vorstellungen der kommenden Welt, aber keine kanonische Festlegung ihrer Beschaffenheit; d. h., das Judentum kennt keine eindeutige Antwort darauf, was nach dem Tod mit uns geschieht. Die Frage nach dem Leben nach dem Tod wird auch als weniger wesentlich angesehen, als Fragen, die das Leben des Menschen auf Erden und in der Gesellschaft betreffen.
Der jüdische Glaube an eine kommende Welt bedeutet nicht, dass Menschen, die nie von der Tora gehört haben, böse oder sonst minderwertige Menschen sind. 

In [23]:
print_train_example(-2)

Finde die Lösung auf die Problemstellung: In welchem Film wurde der Power Glove durch Product Placement beworben?
Im folgenden Abschnitt:

Der Power Glove ist ein Datenhandschuh für den rechten Arm. Mittels am TV anzubringender Sensoren kann man durch Bewegungen des Armes das Spiel steuern. Für zahlreiche Titel war bereits eine Steuerung programmiert, für viele muss man diese jedoch noch selbst kalibrieren. Das Spiel Super Gloveball war speziell auf den Power Glove ausgerichtet, ansonsten hielt sich der praktische Nutzen sehr in Grenzen. In der Computerspielkultur gelangte er zu zweifelhafter Berühmtheit, die sich auf eines der frühesten Beispiele für einen „Hype-Backlash“ zurückführen lässt: Nintendo ließ den Powerglove (neben vielen anderen allzu offensichtlichen Produktplatzierungen) in dem Hollywood-Blockbuster ''Joy Stick Heroes'' (englisch ''The Wizard'') vor dem eigentlichen Verkaufsstart bewerben. Die Dissonanz zwischen Werbeversprechen und Realität und die unfreiwillige Komik 

In [24]:
print_train_example(-1)

Gegeben ist folgender Textbereich:

Manche Kritiken gehen auch auf die Produktplatzierung im Film ein:
So schrieb Lea Hermann für ''Focus'': „Auf die Nerven geht irgendwann die Schleichwerbung im ''Schlussmacher'', die eigentlich gar nichts mehr mit schleichen zu tun hat. Äußerst offensichtlich sind die Produkte platziert. Mal liegt ein Schokoriegel gut erkennbar neben Paul im Bett, mal fallen Dosen eines bekannten Energy-Drink-Herstellers aus dem Auto. Oder man denkt, es läuft Werbung, wenn Paul gefühlte fünf Minuten auf das Display seines Smartphones mit sehr gut erkennbaren Marken-Logo starrt.“
Die ''Welt'' befand etwas nüchtern: „Herausragend ist nur das allzu auffällige Product Placement.“ In einem Interview mit der Neuen Osnabrücker Zeitung erklärt Schweighöfer dazu: „Es stimmt natürlich, dass Bahlsen unser Product-Placement-Partner war. Ein Super-Deal, weil wir auch weiterhin zusammenarbeiten – in einem Spot, der sowohl die Kekse als auch unseren Film bewirbt. Das sind die Spiel

In [25]:
def print_test_example(index):
    print(inpus_test[index])
    print(outputs_test[index])

In [26]:
print_test_example(0)

Gegeben ist folgender Abschnitt:

Innerhalb der Alpen ist das französische Grenoble die größte Stadt, gefolgt von Innsbruck in Österreich sowie von Trient und Bozen in Italien. In der Schweiz liegen Chur, Thun und Lugano in den Alpen. Weitere Alpenstädte in Österreich sind Klagenfurt und Villach, sowie im Rheintal Bregenz, Dornbirn und Feldkirch. Ferner zu nennen ist Vaduz, die Hauptstadt Liechtensteins. Die höchste Stadt der Alpen (und Europas) ist das schweizerische Davos.
In direkter Alpenrandlage ist Wien die weitaus größte Stadt, gefolgt von Genf (Schweiz) und Nizza (Frankreich). Weitere wichtige Städte sind – von Ost nach West – Maribor (Slowenien), Graz (Österreich), Ljubljana (Slowenien), Udine (Italien), Salzburg (Österreich), Vicenza (Italien), Verona (Italien), Brescia (Italien), Bergamo (Italien), St. Gallen (Schweiz), Lecco (Italien), Como (Italien), Varese (Italien), Luzern (Schweiz), Savona (Italien), Biella (Italien), San Remo (Italien), Cuneo (Italien), Bern (Schweiz) 

In [27]:
print_test_example(1)

Ich habe eine Anfrage zu folgendem Abschnitt:

Der Großglockner ist Teil des ''Glocknerkamms'', eines Gebirgskamms der Glocknergruppe (Österreichische Zentralalpen), der am Eiskögele in südöstlicher Richtung vom Alpenhauptkamm abzweigt und dort die Grenze zwischen den Bundesländern Tirol im Südwesten und Kärnten im Nordosten bildet. Diese Grenze ist auch die Wasserscheide zwischen dem Kalser Tal mit seinen Seitentälern auf der Osttiroler und dem Mölltal mit der Pasterze auf der Kärntner Seite. Die Gegend um den Berg ist außerdem seit 1986 Bestandteil des ''Sonderschutzgebietes Großglockner-Pasterze'' innerhalb des Nationalparks Hohe Tauern.
Der Großglockner ist der höchste Berg der Alpen östlich der 175 km entfernten Ortlergruppe und weist damit nach dem Mont Blanc die zweitgrößte geografische Dominanz aller Berge der Alpen auf. Auch seine Schartenhöhe ist mit 2.424 Metern nach dem Montblanc die zweitgrößte aller Alpengipfel. Somit ist der Berg eine der eigenständigsten Erhebungen der 

In [28]:
print_test_example(-2)

Finde die Antwort auf die Frage: Mit welchem Antrieb werden Zahnstangenaufzüge betrieben?
Im folgenden Text:

Bei einem Zahnstangenaufzug ist die Aufzugskabine mit einem eigenen Antrieb ausgestattet. Der Antrieb kann durch einen Elektromotor oder einen Verbrennungsmotor nach dem Zahnstangen/Ritzelprinzip erfolgen.
Zahnstangenaufzüge werden unter anderem als Bauaufzüge, Kranführeraufzüge, Rettungsaufzüge oder Wartungsaufzüge eingesetzt, um Material und Personen zu transportieren. Sie werden auch in abgespannten Sendemasten oder ähnlichen Konstruktionen installiert, um die Flugsicherheitslampen oder andere Anlagenteile leichter warten zu können. Beispiele für installierte Zahnstangenaufzüge im Wartungsbereich sind die Tragmasten der Elbekreuzung 2 oder der Sendemast des WDR in Velbert-Langenberg mit Benzinmotor. Als Kranführeraufzüge beim Erzumschlager Hansaport in Hamburg oder als Rettungsaufzüge für die Feuerwehr beim Eisenbahntunnel Zürich–Thalwil. Vielen ist er auch bekannt aus Rundg

In [29]:
print_test_example(-1)

Ich habe eine Fragestellung zu folgendem Text:

In modernen holzverarbeitenden Betrieben wird überwiegend mit motorisiertem Werkzeug gearbeitet. Es kommen Maschinen mit Verbrennungsmotor, Elektromotor und pneumatischen Antrieben zum Einsatz. Durch pneumatischen Antrieb wird Funkenschlag vermieden, durch den eine Staubexplosion ausgelöst werden kann. Maschinen mit Verbrennungsmotor können nur außerhalb geschlossener Räume betrieben werden. Bis heute werden vereinzelt Wasserräder zum Antrieb genutzt deren Kraft über Transmission auf die Maschinen übertragen wird.
Alle Maschinen gibt es in verschiedenen Ausführungen und Formen, je nach Anforderung als stationäre Einrichtung oder als Handmaschine.

Die Fragestellung lautet: Mit welchem Antrieb werden Zahnstangenaufzüge betrieben?
Bitte finde im Text die Lösung und gib präzise nur diese Textstelle wieder.
Informiere mich mit "Keine Lösung gefunden.", wenn nichts gefunden wurde.
Keine Lösung gefunden.


# Save Data

In [30]:
!mkdir -p instruct_data

In [31]:
train_df = pd.DataFrame({"input": inpus_train, "output": outputs_train})
train_df["uuid"] = [uuid.uuid4() for _ in range(len(train_df))]
train_df

,input,output,uuid
0,Finde die Antwort auf die Frage: Wie viele chr...,75 % der befragten Katholiken sowie 67 % der P...,2cb73380-60c9-4596-9fc0-083b939ce6df
1,Gegeben ist folgender Text:\n\nDer Glaube an e...,Die Lösung wurde nicht gefunden.,c9c05ddb-e529-4bba-b331-f6536eaf26be
2,Ich habe folgende Anfrage: Wann werden Kleinsp...,"Die Lösung heißt: ""bei der Beleuchtung von rot...",4513527e-79aa-46dd-9810-cc110e5e11d5
3,Gegeben ist folgender Text:\n\nUm einen schnel...,Antwort nicht gefunden.,187dea7e-8cb2-4a6f-96be-30c9a9a8f0f7
4,Finde die Lösung auf die Anfrage: Wie viel Spe...,"Die Lösung auf die Anfrage heißt: ""neben dem M...",cfae09be-6cbe-4191-994f-1457eb5773b6
...,...,...,...
18536,Gegeben ist folgender Text:\n\nSchlacht_um_Iwo...,Die Antwort wurde nicht gefunden.,e476f5ab-078a-4d14-8e5d-cb4c24cb3d9c
18537,Ich habe eine Anfrage zu folgendem Absatz:\n\n...,"""Powhatan-Konföderation"" heißt das Ergebnis.",d65fff96-043b-40ed-863c-1eba898d029e
18538,Ich habe folgende Anfrage: Zu welchem Stammesz...,Die Lösung wurde nicht gefunden.,6dfb3b34-be0a-40f3-8617-c169d71775e9
18539,Finde die Lösung auf die Problemstellung: In w...,"Die Lösung auf die Problemstellung lautet: ""Jo...",8ba4f5bf-60f2-4609-9f5b-467e07e127a3


In [32]:
train_df.to_csv(f"./instruct_data/{DATASET_NAME_AND_VERSION}_train.csv", index=False)

In [33]:
test_df = pd.DataFrame({"input": inpus_test, "output": outputs_test})
test_df["uuid"] = [uuid.uuid4() for _ in range(len(test_df))]
test_df

,input,output,uuid
0,Gegeben ist folgender Abschnitt:\n\nInnerhalb ...,"""Innsbruck"" ist das Ergebnis.",0a276b4a-a03c-415d-9596-0d13a3fe440a
1,Ich habe eine Anfrage zu folgendem Abschnitt:\...,Lösung nicht gefunden.,37e8f20e-41f6-4b81-b3f8-0bfe5baaf0bd
2,Gegeben ist folgender Textbereich:\n\nDer Schu...,"Ende 2017 auf rund 15,8 Milliarden Euro",f6f85cbc-a36c-410e-aa23-61e6340020f3
3,Ich habe eine Fragestellung zu folgendem Absch...,Keine Antwort gefunden.,4e47b6c2-0cf7-432d-859b-3ba90aa579e9
4,Gegeben ist folgender Absatz:\n\nDer USB-Host-...,"""Transferart, Geräteeigenschaften und Busbelas...",01b3ad75-6278-47f2-8eef-b4cc690d791e
...,...,...,...
2045,Ich habe eine Fragestellung zu folgendem Textb...,Die Lösung wurde nicht gefunden.,1a774b9d-e4c7-4940-ae36-379499d3d0f9
2046,Finde die Lösung auf die Anfrage: Wieso sind h...,"""aufgrund der gemeinsamen Kolonialgeschichte"" ...",03e8cfa2-fbba-4e09-8fe8-398317def04a
2047,Finde die Antwort auf die Problemstellung: Wie...,Keine Antwort gefunden.,43cda5e7-3dc3-4986-b06c-2a41f5461265
2048,Finde die Antwort auf die Frage: Mit welchem A...,"Die Antwort ist: ""einen Elektromotor oder eine...",1ea276bf-e81c-4a8b-8a93-3f3464b0130e


In [34]:
test_df.to_csv(f"./instruct_data/{DATASET_NAME_AND_VERSION}_test.csv", index=False)